<a href="https://colab.research.google.com/github/wilwil2468/SRCNN-Pytorch/blob/main/SRCNN-Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Train**

In [1]:
!pip install lmdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 10.7 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/wilwil2468/IRSRCNN.git

Cloning into 'IRSRCNN'...
remote: Enumerating objects: 12534, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (158/158), done.
remote: Total 12534 (delta 96), reused 160 (delta 49), pack-reused 12326 (from 4)
Receiving objects: 100% (12534/12534), 1.12 GiB | 43.76 MiB/s, done.
Resolving deltas: 100% (206/206), done.
Updating files: 100% (12011/12011), done.


In [3]:
%cd /kaggle/working/IRSRCNN

/kaggle/working/IRSRCNN


In [4]:
!git pull

Already up to date.


For “915” (k9–k1–k5) you used hr_crop_size = 21

For “935” (k9–k3–k5) you used hr_crop_size = 19

For “955” (k9–k5–k5) you used hr_crop_size = 17

In [5]:
!python precompute_patches.py dataset/train train_patches_955.lmdb \
    --hr 17 --lr 33 --per-image 20

Image video-24ysbPEGoEKKDvRt6-frame-000000-4C4FHWxwNaMyohLZt.jpg → patches 20
Image video-24ysbPEGoEKKDvRt6-frame-000015-ceXK8kdaSPB6ojqyZ.jpg → patches 40
Image video-24ysbPEGoEKKDvRt6-frame-000030-JqzDhfF6nR2wLt5dh.jpg → patches 60
Image video-24ysbPEGoEKKDvRt6-frame-000045-7nyNoWkZdoYr7CXa8.jpg → patches 80
Image video-24ysbPEGoEKKDvRt6-frame-000060-RezTnP6tKcPngNX5L.jpg → patches 100
Image video-24ysbPEGoEKKDvRt6-frame-000075-QiHFeD5XJCmhf4oQu.jpg → patches 120
Image video-24ysbPEGoEKKDvRt6-frame-000090-texZDfwT9iwYNRRgG.jpg → patches 140
Image video-24ysbPEGoEKKDvRt6-frame-000105-dGhhn5isEciBcm4Kh.jpg → patches 160
Image video-24ysbPEGoEKKDvRt6-frame-000120-wL5LL3hey6HeprLEt.jpg → patches 180
Image video-24ysbPEGoEKKDvRt6-frame-000135-doBTjevToEjCAXbrz.jpg → patches 200
Image video-24ysbPEGoEKKDvRt6-frame-000150-mCb5xZ3NsykJTySm7.jpg → patches 220
Image video-24ysbPEGoEKKDvRt6-frame-000165-QgFfTTjKQ3vBoek3y.jpg → patches 240
Image video-24ysbPEGoEKKDvRt6-frame-000180-ioP2AstLukvvS

In [6]:
!python precompute_patches.py dataset/validation valid_patches_955.lmdb \
    --hr 17 --lr 33 --per-image 5

Image video-57kWWRyeqqHs3Byei-frame-000816-b6tuLjNco8MfoBs3d.jpg → patches 5
Image video-57kWWRyeqqHs3Byei-frame-000831-dsrhuE2hoMsERqDZT.jpg → patches 10
Image video-57kWWRyeqqHs3Byei-frame-000846-9A58EHiNDm7amc4Mq.jpg → patches 15
Image video-57kWWRyeqqHs3Byei-frame-000861-FnEJHM4xCbiFuLD5q.jpg → patches 20
Image video-57kWWRyeqqHs3Byei-frame-000876-JtZgCe4rmRSco7TRQ.jpg → patches 25
Image video-57kWWRyeqqHs3Byei-frame-000891-yMxkpA3QbaX4J8mQd.jpg → patches 30
Image video-57kWWRyeqqHs3Byei-frame-000906-xFrHgioZxr6Mwh9Zq.jpg → patches 35
Image video-57kWWRyeqqHs3Byei-frame-000921-NMdfjzsYjymetcQH8.jpg → patches 40
Image video-57kWWRyeqqHs3Byei-frame-000936-cmMSinNmaSStfEFfZ.jpg → patches 45
Image video-57kWWRyeqqHs3Byei-frame-000951-SYQNvB8sKXjZbEPsp.jpg → patches 50
Image video-57kWWRyeqqHs3Byei-frame-000966-bDLjKDs3TPANvseae.jpg → patches 55
Image video-57kWWRyeqqHs3Byei-frame-000981-2ZwyW7ux6MurM56Nv.jpg → patches 60
Image video-57kWWRyeqqHs3Byei-frame-000996-MupkM9Y3jkikMTQvh.jpg 

In [7]:
# train SRCNN-955 model
!rm -rf dataset/*.npy
!python train.py  --steps=300000                    \
                  --architecture="955"              \
                  --batch_size=128                \
                  --save-best-only=1                \
                  --save-every=1000                 \
                  --save-log=0                      \
                  --ckpt-dir="checkpoint/SRCNN955"  \
                  --num-worker=4

/kaggle/working/IRSRCNN/model.py:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.ckpt_man = torch.load(ckpt_path)
/kaggle/working/IRSRCNN/model.py:53: FutureWarning: 

# **Test**

In [8]:
# Test on Set5
!python test.py --scale=2 --architecture=955 --ckpt-path="default"
!python test.py --scale=3 --architecture=955 --ckpt-path="default"
!python test.py --scale=4 --architecture=955 --ckpt-path="default"

/kaggle/working/IRSRCNN/model.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(filepath, map_location=torch.device(self.device))
25.480439758300783
/kaggle/wo

In [9]:
# Test on FLIR data
!python test-ir.py --scale=2 --architecture=955 --ckpt-path="default"
!python test-ir.py --scale=3 --architecture=955 --ckpt-path="default"
!python test-ir.py --scale=4 --architecture=955 --ckpt-path="default"

/kaggle/working/IRSRCNN/model.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(filepath, map_location=torch.device(self.device))
32.82620493570963
/kaggle/wor

# **Demo**

In [10]:
!python demo.py --image-path="dataset/test1.png" \
                --architecture="955"             \
                --ckpt-path="default"            \
                --scale=4

/kaggle/working/IRSRCNN/model.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(filepath, map_location=torch.device(self.device))
